In [1]:
import numpy as np
import pandas as pd
from calendar import monthrange
from typing import Union, List, Dict
from libraries.IndexesExtraction import IndexesExtraction

In [2]:
periods = [str(i) for i in list(range(1964, 2022))]

In [3]:
classification_rules = {
    'fraca':            np.array(range(-31, -51, -1)),
    'moderada':         np.array(range(-51, -101, -1)),
    'intensa':          np.array(range(-101, -251, -1)),
    'super_intensa':    np.array(range(-251, -1001, -1)),
}

# indexes = IndexesExtraction(periods=periods)
# indexes.make_classification(classification_rules, classification_by_column='Dst_index')

In [6]:
def save_processed_data(df, folder_name: str, name: str):
    processed_data = pd.HDFStore(path=f'/home/gabriel/tcc/OMNI/processed_data/{name}.h5')
    processed_data.append('df', df)
    processed_data.close()
    print(f'Seus dados foram salvos em: processed_data/{name}.h5\n')

# save_processed_data(df=df_test, folder_name='OMNI', name='full_data')

def load_processed_data(folder_name:str, name: str):
    processed_data = pd.HDFStore(path=f'/home/gabriel/tcc/OMNI/processed_data/{name}.h5')
    df = processed_data['df']
    processed_data.close()
    return df


In [7]:

data = load_processed_data(folder_name='OMNI', name='full_data')

In [8]:
df_test = data.copy()

In [9]:
df_test

,date,Dst_index,Kp_index,B_scalar,Bz_GSM,classification
6,2001-06-07,-36.0,57.0,12.5,-6.1,fraca
11,2001-06-12,-61.0,53.0,19.3,-11.8,moderada
17,2001-06-18,-33.0,57.0,8.4,-4.1,fraca
26,2001-06-27,-46.0,37.0,16.3,-7.1,fraca
41,1986-12-12,-39.0,43.0,999.9,999.9,fraca
...,...,...,...,...,...,...
21176,1996-09-22,-37.0,27.0,7.5,-3.2,fraca
21177,1996-09-23,-42.0,50.0,12.2,-7.5,fraca
21180,1996-09-26,-38.0,50.0,9.3,-5.7,fraca
21183,1996-09-29,-42.0,43.0,9.7,-4.8,fraca


In [10]:
# df_test.drop(df_test[df_test['Dst_index'] == 999.9].index, inplace=True)
df_test.drop(df_test[df_test['B_scalar'] == 999.9].index, inplace=True)
# df_test.drop(df_test[df_test['Bz_GSM'] == 999.9].index, inplace=True)
# df_test.drop(df_test[df_test['Dst_index'] == 999.9].index, inplace=True)

In [11]:
def class_count(df):
    count = {"fraca": 0, "moderada": 0, "intensa": 0, "super_intensa": 0}
    for i in range(1, len(df)):
        count[df['classification'].iloc[i]]+=1
    print(count)

In [12]:
class_count(df_test)

{'fraca': 2241, 'moderada': 1105, 'intensa': 216, 'super_intensa': 15}
